Importar librerías

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

Crear las rutas de acceso

In [2]:
# Ruta base donde se descomprimió el dataset Darwin Core Archive
folder = Path.home() / "Downloads" / "wildlife-sounds-birds_dwca"

# Archivos de entrada
occ_path = (folder / "Occurrence.txt").as_posix()
mult_path = (folder / "Multimedia.txt").as_posix()

# Archivo de salida (dataset combinado)
out_csv = (Path.home() / "Downloads" / "birds_spain_joined.csv").as_posix()

print("Archivo Occurrence:", occ_path)
print("Archivo Multimedia:", mult_path)
print("Archivo de salida:", out_csv)

Archivo Occurrence: C:/Users/anto-/Downloads/wildlife-sounds-birds_dwca/Occurrence.txt
Archivo Multimedia: C:/Users/anto-/Downloads/wildlife-sounds-birds_dwca/Multimedia.txt
Archivo de salida: C:/Users/anto-/Downloads/birds_spain_joined.csv


Crear la conexión DuckDB

In [3]:
# Conexión a DuckDB en memoria
con = duckdb.connect(database=":memory:")

print("Conexión establecida correctamente.")

Conexión establecida correctamente.


Cargamos Ocurrence, filtros para España y multimedia

In [4]:
# Cargar Occurrence.txt y filtrar registros donde country = 'Spain'
con.execute(f"""
    CREATE OR REPLACE TABLE occurrence_spain AS
    SELECT *
    FROM read_csv_auto('{occ_path}')
    WHERE country = 'Spain';
""")

print("Tabla occurrence_spain creada correctamente.")


Tabla occurrence_spain creada correctamente.


In [5]:
# Cargar Multimedia.txt completo
con.execute(f"""
    CREATE OR REPLACE TABLE multimedia AS
    SELECT *
    FROM read_csv_auto('{mult_path}');
""")

print("Tabla multimedia creada correctamente.")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Tabla multimedia creada correctamente.


Inspeccionar columnas para validar el JOIN

In [6]:
# Revisar columnas de occurrence_spain
print("Columnas de occurrence_spain:")
display(con.execute("PRAGMA table_info(occurrence_spain);").fetchdf())

# Revisar columnas de multimedia
print("Columnas de multimedia:")
display(con.execute("PRAGMA table_info(multimedia);").fetchdf())


Columnas de occurrence_spain:


,cid,name,type,notnull,dflt_value,pk
0,0,id,VARCHAR,False,None,False
1,1,occurrenceID,VARCHAR,False,None,False
2,2,catalogNumber,VARCHAR,False,None,False
3,3,basisOfRecord,VARCHAR,False,None,False
4,4,collectionCode,VARCHAR,False,None,False
5,5,dynamicProperties,VARCHAR,False,None,False
6,6,otherCatalogNumbers,VARCHAR,False,None,False
7,7,genus,VARCHAR,False,None,False
8,8,specificEpithet,VARCHAR,False,None,False
9,9,infraspecificEpithet,VARCHAR,False,None,False


Columnas de multimedia:


,cid,name,type,notnull,dflt_value,pk
0,0,CoreId,VARCHAR,False,None,False
1,1,associatedObservationReference,VARCHAR,False,None,False
2,2,Identifier,VARCHAR,False,None,False
3,3,type,VARCHAR,False,None,False
4,4,Rating,BIGINT,False,None,False
5,5,rightsHolder,VARCHAR,False,None,False
6,6,creator,VARCHAR,False,None,False
7,7,accessURI,VARCHAR,False,None,False
8,8,format,VARCHAR,False,None,False
9,9,variantLiteral,VARCHAR,False,None,False


Ejecutamos el JOIN entre Occurence y Multimedia

In [7]:
# Unión entre Occurrence (id) y Multimedia (CoreId)
# Además añadimos columnas multimedia relevantes: Identifier (URL), type, Rating

con.execute("""
    CREATE OR REPLACE TABLE birds_spain AS
    SELECT
        o.*,
        m.Identifier AS audio_url,
        m.type       AS media_type,
        m.Rating     AS media_rating
    FROM occurrence_spain o
    LEFT JOIN multimedia m
        ON o.id = m.CoreId;
""")

print("Tabla birds_spain creada correctamente.")


Tabla birds_spain creada correctamente.


Exportar el dataset combinado a CSV

In [8]:
# Exportar el dataset final
con.execute(f"""
    COPY birds_spain
    TO '{out_csv}'
    WITH (HEADER, DELIMITER ',');
""")

print("CSV final exportado correctamente en:", out_csv)


CSV final exportado correctamente en: C:/Users/anto-/Downloads/birds_spain_joined.csv


Cargar el CSV en pandas para el análisis

In [9]:
# Cargar dataset combinado en pandas para continuar el análisis
df_spain = pd.read_csv(out_csv)

print("Dimensiones del dataframe final:", df_spain.shape)
df_spain.head()


C:\Users\anto-\AppData\Local\Temp\ipykernel_38504\4057851898.py:2: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_spain = pd.read_csv(out_csv)


Dimensiones del dataframe final: (369321, 40)


,id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license,audio_url,media_type,media_rating
0,902066@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC902066,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Olympus"",""microphone"":""AOM...",NaN,Gelochelidon,nilotica,NaN,...,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Christian Kerihuel,CC BY-NC,https://xeno-canto.org/sounds/uploaded/VWWRQKF...,StillImage,NaN
1,902066@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC902066,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Olympus"",""microphone"":""AOM...",NaN,Gelochelidon,nilotica,NaN,...,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Christian Kerihuel,CC BY-NC,https://xeno-canto.org/sounds/uploaded/VWWRQKF...,StillImage,NaN
2,902066@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC902066,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Olympus"",""microphone"":""AOM...",NaN,Gelochelidon,nilotica,NaN,...,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Christian Kerihuel,CC BY-NC,https://xeno-canto.org/sounds/uploaded/VWWRQKF...,StillImage,NaN
3,902066@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC902066,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Olympus"",""microphone"":""AOM...",NaN,Gelochelidon,nilotica,NaN,...,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Christian Kerihuel,CC BY-NC,https://xeno-canto.org/sounds/uploaded/VWWRQKF...,StillImage,NaN
4,902066@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC902066,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Olympus"",""microphone"":""AOM...",NaN,Gelochelidon,nilotica,NaN,...,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Christian Kerihuel,CC BY-NC,https://xeno-canto.org/sounds/uploaded/VWWRQKF...,Sound,4.0


Comprobación de la calidad de porcentaje de registros donde se ha registrado el audio

In [10]:
# Calcular qué porcentaje de registros tiene audio_url no nulo
porcentaje_audio = df_spain["audio_url"].notna().mean()

print(f"Porcentaje de registros con audio asociado: {porcentaje_audio:.2%}")


Porcentaje de registros con audio asociado: 99.89%


In [11]:
# Revisión rápida de las columnas más relevantes
columnas_interes = [
    "id",
    "scientificName",
    "family",
    "vernacularName",
    "eventDate",
    "latitudeDecimal",
    "longitudeDecimal",
    "audio_url"
]

df_spain[columnas_interes].head()


,id,scientificName,family,vernacularName,eventDate,latitudeDecimal,longitudeDecimal,audio_url
0,902066@XC,Gelochelidon nilotica,Laridae,Gull-billed Tern,2024-05-11,42.2087,3.0994,https://xeno-canto.org/sounds/uploaded/VWWRQKF...
1,902066@XC,Gelochelidon nilotica,Laridae,Gull-billed Tern,2024-05-11,42.2087,3.0994,https://xeno-canto.org/sounds/uploaded/VWWRQKF...
2,902066@XC,Gelochelidon nilotica,Laridae,Gull-billed Tern,2024-05-11,42.2087,3.0994,https://xeno-canto.org/sounds/uploaded/VWWRQKF...
3,902066@XC,Gelochelidon nilotica,Laridae,Gull-billed Tern,2024-05-11,42.2087,3.0994,https://xeno-canto.org/sounds/uploaded/VWWRQKF...
4,902066@XC,Gelochelidon nilotica,Laridae,Gull-billed Tern,2024-05-11,42.2087,3.0994,https://xeno-canto.org/sounds/uploaded/VWWRQKF...
